# Práctica 5

In [281]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

## Ejercicio 1: Regresión lineal regularizada

In [282]:
mat = loadmat('ex5data1.mat')
X = mat.get("X")
y = mat.get("y")
Xval = mat.get("Xval")
yval = mat.get("yval")

print(X.shape, y.shape)
print(Xval.shape, yval.shape)
print(np.mean(X), np.std(X))
print(np.mean(y), np.std(y))

(12, 1) (12, 1)
(21, 1) (21, 1)
-5.085426348834809 28.68873075847896
11.217589325366376 12.492955274415026


In [283]:
def coste_vec(Theta, X, y):   
    # añadimos una columna de 1's a la X
    m = np.shape(X)[0]
   # X = np.hstack([np.ones([m, 1]), X])
    H = np.dot(X, Theta.T)
    #H = np.dot(Theta, X)
    Aux = (H - y) ** 2
    return Aux.sum() / (2 * len(X))

In [284]:
def coste_reg(Theta, X, y, Lambda):
    Theta = Theta.reshape((Theta.shape[0],1))
    Term1 = coste_vec(Theta, X, y)
    Term2 = (Lambda/(2*len(y))) * np.sum(Theta**2)
    cost = Term1 + Term2
    return cost 

In [285]:
def gradiente_vec(Theta, X, Y):
    m = np.shape(X)[0]
    H = np.dot(X, Theta.T)
    Aux = (H - Y)
    grad = np.dot(Aux.T, X) * (1/m)
    return grad

In [286]:
def gradiente_reg(Theta, X, Y, Lambda):
    Theta = Theta.reshape((Theta.shape[0],1))
    item1 = gradiente_vec(Theta, X, Y).T
    item2  = (Lambda/len(y))*np.r_[[np.zeros(Theta.shape[1])], Theta[1:]]
    #item2  = (Lambda/len(y))*np.r_[[0], Theta[1:]]
    grad = item1 + item2
    grad = grad.flatten()
    return grad

In [287]:
def coste_grad_reg(Theta, X, Y, Lambda):
    coste = coste_reg(Theta, X, y, Lambda)
    grad = gradiente_reg(Theta, X, Y, Lambda)
    return coste, grad

In [288]:
theta = np.ones((2,1))
#theta = np.ones(2)
m=len(y)
X_1 = np.hstack([np.ones([m, 1]), X])
coste, grad = coste_grad_reg(theta, X_1, y, 1)
print(coste, grad)

ValueError: shapes (12,2) and (1,2) not aligned: 2 (dim 1) != 1 (dim 0)

In [ ]:
print(X_1.shape)
print(theta.shape)

In [ ]:
Lambda = 0
theta = np.ones((2,1))
result = opt.fmin_tnc(func=coste_reg, x0=theta, fprime=gradiente_reg, args=(X_1,y,Lambda))
Theta_opt=result[0]
coste_opt = coste_reg(Theta_opt, X_1, y, Lambda)
print('Coste = {}'.format(coste_opt))

In [ ]:
print(Theta_opt.shape)

In [ ]:
def plot_H(X, Y, theta):
    plt.plot(X, Y, "x")
    theta0 = theta[0]
    theta1 = theta[1]
    min_x = min(X)
    max_x = max(X)
    min_y = theta0 + theta1 * min_x
    max_y = theta0 + theta1 * max_x
    plt.plot([min_x, max_x], [min_y, max_y])
    plt.savefig("resultado.pdf")

In [ ]:
plot_H(X, y, Theta_opt)

## Ejercicio 2: Curvas de aprendizaje

In [ ]:
def curva_aprendizaje(theta, X, y, Lambda):
    m = len(y)
    X_1 = np.hstack([np.ones([m, 1]), X])
    muestras, J_train, J_val = [], [], []
    for i in range(1, 11, 1):
        X_train_aux = X_1[:i, :]
        y_train_aux = y[:i]
        
        result = opt.fmin_tnc(func=coste_reg, x0=theta, fprime=gradiente_reg, args=(X_train_aux,y_train_aux,Lambda))
        theta_aux = result[0]
        J_train.append(coste_reg(theta_aux,X_train_aux,y_train_aux,Lambda))
        J_val.append(coste_reg(theta_aux, X_1, y,Lambda))
        muestras.append(y_train_aux.shape[0])
    return muestras, J_train, J_val

In [ ]:
def plot_curva(muestras, J_train, J_val):
    plt.figure()
    plt.plot(muestras, J_train, label='Train')
    plt.plot(muestras, J_val, label='Cross Validation')
    plt.legend()
    plt.title('Learning curve for linear regression')
    plt.xlabel('Number of training examples')
    plt.ylabel('Error')
    plt.show()

In [ ]:
print(X)
print(X.shape)

In [ ]:
muestras, J_train, J_val = curva_aprendizaje(theta, Xval, yval, 0)
plot_curva(muestras, J_train, J_val)

## Ejercicio 3: Regresión polinomial

In [ ]:
def expand_X(X, p):
    m=X.shape[0]
    X_p = np.zeros((m,p))
    for i in range(p):
        dim = i+1
        X_p[:, i] = X[:, 0] ** (i + 1)
 
    return X_p
    

In [ ]:
X_8 = expand_X(X,8)

In [ ]:
print(X_8)
print(X_8.shape)

In [ ]:
def normalizer(X):
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)
    X_norm = (X-mu)/sigma
    return X_norm, mu, sigma

In [ ]:
X_8_norm, mu, sigma = normalizer(X_8)
print(X_8_norm)
print(X_8_norm.shape)

In [ ]:
print(mu)
print(mu.shape)

print(sigma)
print(sigma.shape)

In [ ]:
#Añadimos columna de 1's a los datos de entrenamiento X_8_norm
m = X_8_norm.shape[0]
X_1 = np.hstack([np.ones([m, 1]), X_8_norm])

In [ ]:
Lambda = 0
theta = np.ones((9,1))
result = opt.fmin_tnc(func=coste_reg, x0=theta, fprime=gradiente_reg, args=(X_1,y,Lambda))
Theta_opt=result[0]
coste_opt = coste_reg(Theta_opt, X_1, y, Lambda)
print('Coste = {}'.format(coste_opt))

In [ ]:
print(Theta_opt)

In [ ]:
def plot_poli(X, y, theta, mu, sigma):
    plt.plot(X, y, "x")
    
    x1_min, x1_max = X[:, 0].min(), X[:, 0].max()
    xx1 = np.linspace(x1_min, x1_max)
    
    xx1=xx1.reshape((xx1.shape[0],1))
    
    theta=theta.reshape((theta.shape[0],1))
    
    X_8 = expand_X(xx1,8)
    
    X_8_norm = (X_8 - mu)/sigma
    X_8_norm_1 = np.hstack([np.ones([X_8_norm.shape[0], 1]), X_8_norm])

    
    h=0
    h = np.matmul(X_8_norm_1,theta)
        # 50x8 x 8, -> 50x1 -> 50,

    plt.plot(xx1, h)  
    plt.title('Polynomial regression (lambda = 0)')
    plt.xlabel('Change in water level (x)')
    plt.ylabel('Water lowing out of the dam (y)')
    plt.show()


In [ ]:
plot_poli(X, y, Theta_opt, mu, sigma)

In [ ]:
def curva_aprendizaje_poli(theta, X, y, Lambda, sigma, mu):
    m = len(y)
    X_1 = np.hstack([np.ones([m, 1]), X])
    Xval_8_norm, aux1, aux2 = normalizer(expand_X(X,8))
    
    muestras, J_train, J_val = [], [], []
    for i in range(1, 13, 1):
        X_train_aux = X_1[:i, :]
        y_train_aux = y[:i]
        
        X_8 = expand_X(X_train_aux,8)
        #X_8_norm, aux3, aux4 = normalizer(X_8)
        X_8_norm = ((X_8 - mu)/sigma)
        X_train_aux = X_8_norm.T
        #X_train_aux = np.hstack([np.ones([X_8_norm.shape[0], 1]), X_8_norm])
        
        result = opt.fmin_tnc(func=coste_reg, x0=theta, fprime=gradiente_reg, args=(X_train_aux,y_train_aux,Lambda))
        theta_aux = result[0]
        J_train.append(coste_reg(theta_aux,X_train_aux,y_train_aux,Lambda))
        J_val.append(coste_reg(theta_aux, Xval_8_norm, y,Lambda))
        muestras.append(y_train_aux.shape[0])
    return muestras, J_train, J_val

In [ ]:
print(Xval)
print(Xval.shape)

In [ ]:
theta = np.ones((8+2,1))
muestras, J_train, J_val = curva_aprendizaje_poli(theta, Xval, yval, 0, sigma, mu)
plot_curva(muestras, J_train, J_val)